### Análise da Rede Completa: Estrutura Geral do Debate

**Objetivo:** Mapear a topologia completa da conversa para identificar a estrutura fundamental do debate, testando a hipótese de polarização.

**Grafo de Análise:** `G_giant` (O maior componente conectado da rede de interações, representando a conversa principal).

**Questão de Pesquisa Específica:** A rede de interações nos comentários se organiza em comunidades distintas e com pouca comunicação entre si, caracterizando um debate polarizado?

**Passos da Análise (Metodologia):**
* Carregar o grafo `G_giant` no ambiente de análise (NetworkX).
* Executar o algoritmo de detecção de comunidades **Louvain** para encontrar a partição ótima da rede.
* Calcular a **Modularidade (Q)** da partição encontrada para quantificar a força da divisão.
* Calcular métricas de centralidade (Grau, Intermediação) para todos os nós de `G_giant` para identificar os atores mais importantes no debate geral.

**Resultados Esperados (Saídas):**
* **Métrica 1:** Pontuação de Modularidade (Q).
* **Métrica 2:** Número de comunidades principais identificadas.
* **Tabela 1:** Lista dos "Top 10" nós mais centrais do grafo geral e suas respectivas pontuações.
* **Figura 1:** Visualização do grafo `G_giant` completo, com nós coloridos por comunidade e dimensionados por centralidade (a ser gerada posteriormente, possivelmente com Gephi ou um código de visualização aprimorado).

In [20]:
# SETUP: DEFINIÇÃO DOS CAMINHOS DO PROJETO

from pathlib import Path

PROJECT_ROOT = Path.cwd().parent

DATA_RAW_DIR = PROJECT_ROOT / 'data' / 'raw'
DATA_PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
FIGURES_DIR = PROJECT_ROOT / 'figures'
SCRIPTS_DIR = PROJECT_ROOT / 'scripts'

print(f"Raiz do projeto definida em: {PROJECT_ROOT}")
print(f"Pasta de dados brutos: {DATA_RAW_DIR}")

Raiz do projeto definida em: /home/henrique/Documentos/UFV2025/RedesComplexas/ProjetoRedes
Pasta de dados brutos: /home/henrique/Documentos/UFV2025/RedesComplexas/ProjetoRedes/data/raw


In [ ]:

# CRIAÇÃO E ANÁLISE DA REDE

import pandas as pd
import networkx as nx
from networkx.algorithms import community
from IPython.display import display

# ETAPA DE CRIAÇÃO DO GRAFO

# Uso da variável DATA_RAW_DIR definida na célula de setup de caminhos.
arquivo_de_arestas = DATA_RAW_DIR / 'rede_usuarios_arestas.csv'

try:
    df_rede = pd.read_csv(arquivo_de_arestas)
    G = nx.from_pandas_edgelist(
        df_rede, 
        source='source',
        target='target',
        edge_attr='peso',
        create_using=nx.Graph()
    )
    print(f"Grafo 'G' inicial criado com {G.number_of_nodes()} nós e {G.number_of_edges()} arestas.")
    
    # FILTRO: Focar apenas no Componente Gigante 
    connected_components = sorted(nx.connected_components(G), key=len, reverse=True)

    grafo_completo = G.subgraph(connected_components[0]).copy() 
    
    print(f"Grafo 'grafo_completo' isolado com {grafo_completo.number_of_nodes()} nós.")

except FileNotFoundError:
    print(f"ERRO: O arquivo '{arquivo_de_arestas}' não foi encontrado!")
    raise
except Exception as e:
    print(f"Ocorreu um erro ao tentar criar o grafo: {e}")
    raise


# ETAPA DE ANÁLISE DO GRAFO

comunidades = community.louvain_communities(grafo_completo, weight='peso', seed=42)
modularidade = community.modularity(grafo_completo, comunidades, weight='peso')
grau = dict(grafo_completo.degree(weight='peso'))
intermediacao = nx.betweenness_centrality(grafo_completo, weight='peso', normalized=True)
df_centralidade = pd.DataFrame({'grau': pd.Series(grau), 'intermediacao': pd.Series(intermediacao)})


# GERAÇÃO DOS RESULTADOS

print("\n" + "="*60)
print("RESULTADOS DA ANÁLISE DA REDE COMPLETA")
print("="*60)
print(f"\nPontuação de Modularidade (Q): {modularidade:.4f}")
print(f"Número de Comunidades: {len(comunidades)}")
print("\nTop 10 Usuários por Centralidade de Grau (Mais Ativos):")
display(df_centralidade.sort_values(by='grau', ascending=False).head(10))
print("\nTop 10 Usuários por Centralidade de Intermediação (Melhores Pontes):")
display(df_centralidade.sort_values(by='intermediacao', ascending=False).head(10))


# ETAPA DE EXPORTAÇÃO PARA O GEPHI

partition = {node: i for i, comm in enumerate(comunidades) for node in comm}
nx.set_node_attributes(grafo_completo, partition, 'comunidade')
nx.set_node_attributes(grafo_completo, grau, 'grau_ponderado')

# Uso da variável DATA_PROCESSED_DIR definida na célula de setup.
nome_arquivo_exportacao = DATA_PROCESSED_DIR / 'grafo_completo_gephi.gexf'
nx.write_gexf(grafo_completo, nome_arquivo_exportacao)


Grafo 'G' inicial criado com 3815 nós e 4059 arestas.
Grafo 'grafo_completo' isolado com 2051 nós.

RESULTADOS DA ANÁLISE DA REDE COMPLETA

Pontuação de Modularidade (Q): 0.8897
Número de Comunidades: 36

Top 10 Usuários por Centralidade de Grau (Mais Ativos):


,grau,intermediacao
@Comunista_Enrustida6378,173,0.443929
@adrianvantasset8081,129,0.272365
@ivi8060,101,0.268493
@AugusttoDalzotto,72,0.001086
@SantanaWithout4,62,0.093598
@Loli-iw,56,0.056779
@joelsonlimadeoliveiraolive9924,34,0.055708
@gabrielf.7968,33,0.027353
@Jp-g4cy,32,0.024253
@RethusOFC,29,0.065846



Top 10 Usuários por Centralidade de Intermediação (Melhores Pontes):


,grau,intermediacao
@Comunista_Enrustida6378,173,0.443929
@adrianvantasset8081,129,0.272365
@ivi8060,101,0.268493
@groove6593,5,0.118776
@viniciusalgumacoisa,7,0.112673
@medina.hzn9,6,0.109263
@lulunvs,7,0.106319
@Gustavo-uh8ww,4,0.095820
@oblivion.4ngel,16,0.095109
@SantanaWithout4,62,0.093598


### Figura 1: Visualização da Rede Completa

A imagem abaixo foi gerada com o Gephi e mostra a estrutura de comunidades da rede. Os nós são coloridos por comunidade e dimensionados por seu grau ponderado.

![Visualização da rede completa gerada no Gephi](../figures/grafo_completo.svg)

### 2. Análise do Subgrafo I: O Núcleo Denso de Interações (K-Core)

**Objetivo:** Investigar a dinâmica interna do grupo de usuários mais engajados e coesos da rede, testando a hipótese de que a polarização se intensifica neste subgrupo. Para isso, aplicaremos um filtro k-core, que isola os nós que mantêm um grau mínimo de conexão mútua.

**Grafo de Análise:** `grafo_core` (Subgrafo gerado pela aplicação do filtro **k-core** em `grafo_completo`, com `k` a ser definido, ex: `k=5`).

**Questão de Pesquisa Específica:** A estrutura de interações dentro do núcleo de usuários mais engajados é ainda mais segregada (possui maior modularidade) do que a da rede completa?

**Passos da Análise (Metodologia):**
* Gerar o subgrafo `grafo_core` a partir de `grafo_completo`.
* Executar novamente o algoritmo de detecção de comunidades Louvain, desta vez aplicado apenas sobre o `grafo_core`.
* Calcular a **Modularidade (Q')** da partição encontrada para o subgrafo.
* Recalcular as métricas de centralidade para os nós pertencentes ao `grafo_core` para identificar os líderes deste núcleo.

**Resultados Esperados (Saídas):**
* **Métrica 3:** Pontuação de Modularidade do núcleo (Q').
* **Tabela 2:** Lista dos "Top 10" nós mais centrais *dentro do núcleo*.
* **Figura 2:** Visualização do subgrafo `grafo_core`, com nós coloridos por suas novas comunidades e dimensionados por sua centralidade *local*.

**Interpretação e Comparação:**
* A comparação entre **Q' (modularidade do núcleo) e Q (modularidade do grafo completo)** será a análise central. Se **Q' > Q**, teremos uma forte evidência de que a polarização é exacerbada entre os participantes mais dedicados.
* A comparação da **Tabela 2 com a Tabela 1** (da análise anterior) mostrará se os líderes do debate geral são os mesmos que formam o seu núcleo denso.

In [ ]:
# ANÁLISE DO SUBGRAFO I: NÚCLEO DENSO (K-CORE)

import pandas as pd
import networkx as nx
from networkx.algorithms import community
from IPython.display import display

# ETAPA 1: GERAÇÃO DO SUBGRAFO K-CORE
K_CORE_THRESHOLD = 3

try:
    # Remove self-loops (necessário para o algoritmo k_core)
    grafo_completo_sem_loops = grafo_completo.copy()
    loops = list(nx.selfloop_edges(grafo_completo_sem_loops))
    grafo_completo_sem_loops.remove_edges_from(loops)

    # Gera o subgrafo k-core a partir do grafo completo
    grafo_core = nx.k_core(grafo_completo_sem_loops, k=K_CORE_THRESHOLD)
    
    print(f"Grafo 'grafo_core' (k={K_CORE_THRESHOLD}) criado com {grafo_core.number_of_nodes()} nós e {grafo_core.number_of_edges()} arestas.")

except Exception as e:
    print(f"Ocorreu um erro ao gerar o grafo k-core: {e}")
    raise


# ETAPA 2: ANÁLISE DO SUBGRAFO CORE

comunidades_core = community.louvain_communities(grafo_core, weight='peso', seed=42)
modularidade_core = community.modularity(grafo_core, comunidades_core, weight='peso')
grau_core = dict(grafo_core.degree(weight='peso'))
intermediacao_core = nx.betweenness_centrality(grafo_core, weight='peso', normalized=True)
df_centralidade_core = pd.DataFrame({'grau': pd.Series(grau_core), 'intermediacao': pd.Series(intermediacao_core)})


# ETAPA 3: GERAÇÃO DOS RESULTADOS DO SUBGRAFO

print("\n" + "="*60)
print("RESULTADOS DA ANÁLISE DO NÚCLEO DENSO (K-CORE)")
print("="*60)
print(f"\nPontuação de Modularidade (Q'): {modularidade_core:.4f}")
print(f"Número de Comunidades no núcleo: {len(comunidades_core)}")
print("\nTop 10 Usuários por Centralidade de Grau (Dentro do Núcleo):")
display(df_centralidade_core.sort_values(by='grau', ascending=False).head(10))
print("\nTop 10 Usuários por Centralidade de Intermediação (Dentro do Núcleo):")
display(df_centralidade_core.sort_values(by='intermediacao', ascending=False).head(10))


# ETAPA 4: EXPORTAÇÃO DO SUBGRAFO PARA O GEPHI

partition_core = {node: i for i, comm in enumerate(comunidades_core) for node in comm}
nx.set_node_attributes(grafo_core, partition_core, 'comunidade')
nx.set_node_attributes(grafo_core, grau_core, 'grau_ponderado')

# Uso da variável DATA_PROCESSED_DIR definida na célula de setup.
nome_arquivo_exportacao_core = DATA_PROCESSED_DIR / 'grafo_core_gephi.gexf'
nx.write_gexf(grafo_core, nome_arquivo_exportacao_core)


Grafo 'grafo_core' (k=3) criado com 60 nós e 115 arestas.

RESULTADOS DA ANÁLISE DO NÚCLEO DENSO (K-CORE)

Pontuação de Modularidade (Q'): 0.6893
Número de Comunidades no núcleo: 6

Top 10 Usuários por Centralidade de Grau (Dentro do Núcleo):


,grau,intermediacao
@Jp-g4cy,16,0.161374
@Nioniolion,11,0.063653
@Comunista_Enrustida6378,10,0.286743
@SantanaWithout4,8,0.183587
@GonçaloCardoso-w4s,8,0.143776
@Adiano355,7,0.000000
@Anyh23,7,0.046493
@FXdarlysson,6,0.000000
@EmillyVitóriaAlmeidaDornelles,6,0.031089
@JoaoSantos-i6v,6,0.171189



Top 10 Usuários por Centralidade de Intermediação (Dentro do Núcleo):


,grau,intermediacao
@Comunista_Enrustida6378,10,0.286743
@zéeduxRLK,3,0.202536
@SantanaWithout4,8,0.183587
@JoaoSantos-i6v,6,0.171189
@Jp-g4cy,16,0.161374
@GonçaloCardoso-w4s,8,0.143776
@Miguelaparecidomarcilino,3,0.110754
@ApenasMemes-0,3,0.096892
@EstêvãoRocha-e5l,5,0.079671
@Nioniolion,11,0.063653


### Figura 2: Visualização do Núcleo Denso da Rede (K-Core)

A imagem abaixo, gerada com o Gephi, representa o subgrafo do núcleo denso (`k=3`) da rede. Ela isola apenas os usuários mais engajados e mutuamente conectados. Os nós são coloridos por suas comunidades locais e dimensionados por seu grau ponderado.

![Visualização do núcleo denso da rede (k-core)](../figures/grafo_core.svg)

### 3. Análise do Subgrafo II: A Rede de Autoridades e Influenciadores

**Objetivo:** Isolar e analisar a estrutura de interações entre os usuários mais centrais da rede, a fim de compreender a dinâmica da "elite" da discussão e verificar se a polarização geral se mantém neste grupo.

**Grafo de Análise:** `grafo_autoridades` (Subgrafo induzido pelos **Top N usuários** com a maior **Centralidade de Grau de Entrada / In-Degree** no `grafo_completo`).
*Nota: O Grau de Entrada é usado aqui como um proxy para "autoridade" ou "influência", pois mede quem mais *recebeu* respostas, ou seja, quem mais gerou engajamento a partir de seus comentários originais.*

**Questão de Pesquisa Específica:** Os usuários mais influentes da discussão formam uma rede coesa e interconectada, ou a segregação em comunidades, observada na rede completa, também se reflete na interação entre eles?

**Passos da Análise (Metodologia):**
* Calcular o Grau de Entrada (In-Degree) ponderado para todos os nós no `grafo_completo`.
* Identificar a lista dos N usuários com maior Grau de Entrada (ex: os top 100 ou top 5%).
* Gerar o subgrafo `grafo_autoridades`, contendo apenas esses nós "elite" e as arestas que existem **entre eles**.
* Analisar as propriedades deste subgrafo (número de componentes, densidade) e visualizar sua estrutura.

**Resultados Esperados (Saídas):**
* **Descrição 1:** Propriedades estruturais da rede de autoridades (ex: se ela é conectada ou fragmentada).
* **Figura 3:** Visualização do subgrafo `grafo_autoridades`. Os nós devem ser coloridos de acordo com a sua **comunidade original** (identificada na Análise 1) para que possamos ver se há mistura entre os grupos de elite.

**Interpretação e Comparação:**
* Se o `grafo_autoridades` for **fragmentado em componentes que correspondem às comunidades originais**, isso é uma forte evidência de que as "elites" de cada lado do debate se ignoram completamente.
* Se o `grafo_autoridades` for um **grafo conectado**, isso sugere que os influenciadores dos diferentes lados estão, de fato, engajando em um diálogo (ou confronto) direto. A visualização nos mostrará se as conexões são predominantemente dentro do mesmo grupo de cor ou se cruzam as fronteiras das comunidades.

In [ ]:
# ANÁLISE DO SUBGRAFO II: REDE DE AUTORIDADES E INFLUENCIADORES

import pandas as pd
import networkx as nx
from IPython.display import display

# ETAPA 1: SELEÇÃO DOS NÓS DE MAIOR AUTORIDADE

# Parâmetro: Quantos dos usuários mais importantes queremos analisar?
# Vamos começar com os top 100 mais ativos (maior grau). Você pode ajustar este número.
TOP_N_USERS = 200

# Pega a lista dos top N usuários do DataFrame de centralidade
top_n_grau = df_centralidade.sort_values(by='grau', ascending=False).head(TOP_N_USERS)
top_n_nodes_lista = top_n_grau.index.tolist()

print(f"Selecionando os {len(top_n_nodes_lista)} usuários mais ativos para formar a rede de autoridades.")


# ETAPA 2: GERAÇÃO DO SUBGRAFO DE AUTORIDADES

# Cria o subgrafo contendo apenas os nós da lista e as arestas que existem ENTRE ELES.
grafo_autoridades = grafo_completo.subgraph(top_n_nodes_lista).copy()

print(f"Grafo 'grafo_autoridades' criado com {grafo_autoridades.number_of_nodes()} nós e {grafo_autoridades.number_of_edges()} arestas.")


# ETAPA 3: ANÁLISE ESTRUTURAL DO SUBGRAFO

# A principal pergunta é: essa rede de "elites" é conectada ou fragmentada?
num_componentes = nx.number_connected_components(grafo_autoridades)


# ETAPA 4: GERAÇÃO DOS RESULTADOS DO SUBGRAFO

print("\n" + "="*60)
print("RESULTADOS DA ANÁLISE DA REDE DE AUTORIDADES")
print("="*60)
print(f"\nAnalisando o subgrafo dos {TOP_N_USERS} usuários mais ativos:")
print(f"Número de Componentes Conectados: {num_componentes}")
if num_componentes > 1:
    print(">> Interpretação: A rede de autoridades é FRAGMENTADA. Os influenciadores estão divididos em grupos que não interagem entre si.")
else:
    print(">> Interpretação: A rede de autoridades é CONECTADA. Os influenciadores formam um único grande componente de discussão.")

    
# ETAPA 5: EXPORTAÇÃO DO SUBGRAFO PARA O GEPHI

# Herda as comunidades do grafo completo para manter a consistência de cores
partition_autoridades = {node: partition.get(node) for node in grafo_autoridades.nodes()}
nx.set_node_attributes(grafo_autoridades, partition_autoridades, 'comunidade')

# Calcula o grau ponderado LOCAL (dentro da rede de autoridades)
grau_autoridades = dict(grafo_autoridades.degree(weight='peso'))
nx.set_node_attributes(grafo_autoridades, grau_autoridades, 'grau_ponderado')

# Usamos a variável DATA_PROCESSED_DIR definida na célula de setup.
nome_arquivo_exportacao_autoridades = DATA_PROCESSED_DIR / 'grafo_autoridades_gephi.gexf'
nx.write_gexf(grafo_autoridades, nome_arquivo_exportacao_autoridades)
    

Selecionando os 200 usuários mais ativos para formar a rede de autoridades.
Grafo 'grafo_autoridades' criado com 200 nós e 281 arestas.

RESULTADOS DA ANÁLISE DA REDE DE AUTORIDADES

Analisando o subgrafo dos 200 usuários mais ativos:
Número de Componentes Conectados: 43
>> Interpretação: A rede de autoridades é FRAGMENTADA. Os influenciadores estão divididos em grupos que não interagem entre si.

SUCESSO! Grafo de autoridades salvo em '/home/henrique/Documentos/UFV2025/RedesComplexas/ProjetoRedes/data/processed/grafo_autoridades_gephi.gexf'.


### Figura 3: Visualização da Rede de Autoridades e Influenciadores

A imagem abaixo apresenta o subgrafo dos Top N (`N=100`) usuários mais ativos da rede, gerado com o Gephi. Os nós são coloridos de acordo com suas comunidades originais (da Análise 1) para destacar a interação (ou a falta dela) entre os influenciadores de diferentes grupos. O tamanho dos nós reflete seu grau ponderado dentro deste subgrafo.

![Visualização da rede de autoridades e influenciadores](../figures/grafo_autoridades.svg)